# Wetlease Aylık Süre Performansı

Rapor Pazartesi çekildiğinden, wetleaslere ait dwh ve sched den çekilen datalar geçtiğimiz pazar günü kesişmeli. Kesişen günde manuel ayıklama yapılacaktır. 

1. dwh ta halen havada görülen uçuşlar silinir.

2. dwh ta orijine divert etmiş uçuşlar silinir.

3. dwh ve sched verisinde kesişen günlerde yapılan uçuslar sched ten silinir.

Not: dwh tan çekilen data son ay ile filtrelenmese dahi burada filtrelenecek. Kargo uçuşları ve Servis tipi filtrelenecek.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
# from pivottablejs import pivot_ui
# import ipypivot as pt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
dict_reg = {'VEL':"ABW","LER":"ABW" ,
             'ACG':"74A","ACF":"74A", 
             'MRH':"33K", 'MRI':"33K", 'MRG':"33K",
             'LNV':"RH", 'LNY':"RH", 'LNX':"RH"}

In [3]:
sched = pd.read_excel("sched.xlsx")
sched

c:\Users\E_SUER1\AppData\Local\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,NO,Al,FlNo,OS,Date,Day,Orig,STD (UTC),STA (UTC),,...,A/C,Cfg,Srv,Class,Blkt,Cntxt,Act,Change,Time,By
0,,TK,6011,NaN,2023-12-17,SUN,ALA,13:25:00,18:15:00,,...,33X,33X,F,NaN,04:50:00,Base,NEW,2023-09-11,15:33:00,elvinb
1,,TK,6011,NaN,2023-12-17,SUN,IST,06:40:00,11:55:00,,...,33X,33X,F,NaN,05:15:00,Base,NEW,2023-09-11,15:33:00,elvinb
2,,TK,6012,NaN,2023-12-17,SUN,CGO,20:15:00,02:35:00,1,...,33X,33X,F,NaN,06:20:00,Base,NEW,2023-09-11,15:33:00,elvinb
3,,TK,6013,NaN,2023-12-17,SUN,IST,22:30:00,03:00:00,1,...,74A,ACT,F,NaN,04:30:00,Base,NEW,2023-09-11,15:42:00,elvinb
4,,TK,6058,NaN,2023-12-17,SUN,HKG,05:40:00,13:10:00,,...,33X,33X,F,NaN,07:30:00,Base,NEW,2023-11-22,14:24:00,emirhand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,,TK,6782,NaN,2023-12-31,SUN,ALA,06:10:00,12:30:00,,...,3RH,3RH,H,NaN,06:20:00,Base,NEW,2023-10-24,11:39:00,bilgeka
1044,,TK,6786,NaN,2023-12-31,SUN,IST,23:05:00,04:40:00,1,...,3RH,3RH,H,NaN,05:35:00,Base,NEW,2023-09-11,15:34:00,merveh
1045,,TK,6787,NaN,2023-12-31,SUN,HKG,05:30:00,12:30:00,,...,3RH,3RH,H,NaN,07:00:00,Base,NEW,2023-09-11,15:34:00,merveh
1046,,TK,6787,NaN,2023-12-31,SUN,ALA,14:00:00,20:30:00,,...,3RH,3RH,H,NaN,06:30:00,Base,NEW,2023-09-11,15:34:00,merveh


In [4]:
sched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 22 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   NO         1048 non-null   object        
 1   Al         1048 non-null   object        
 2   FlNo       1048 non-null   object        
 3   OS         0 non-null      float64       
 4   Date       1048 non-null   datetime64[ns]
 5   Day        1048 non-null   object        
 6   Orig       1048 non-null   object        
 7   STD (UTC)  1048 non-null   object        
 8   STA (UTC)  1048 non-null   object        
 9              1048 non-null   object        
 10  Dest       1048 non-null   object        
 11  Own        1048 non-null   object        
 12  A/C        1048 non-null   object        
 13  Cfg        1048 non-null   object        
 14  Srv        1048 non-null   object        
 15  Class      0 non-null      float64       
 16  Blkt       1048 non-null   object        


In [5]:
sched.columns

Index(['NO', 'Al', 'FlNo', 'OS', 'Date', 'Day', 'Orig', 'STD (UTC)',
       'STA (UTC)', ' ', 'Dest', 'Own', 'A/C', 'Cfg', 'Srv', 'Class', 'Blkt',
       'Cntxt', 'Act', 'Change', 'Time', 'By'],
      dtype='object')

# Manipulation in Sched

## FlightNo and BH

In [6]:
sched.rename(columns={' ':"over","A/C":"AC","STA (UTC)":"STA","STD (UTC)":"STD"},inplace=True)
# revise overnight column
sched.over.replace(['  ', ' +1', ' +2'],[0,1,2],inplace=True)

In [7]:
# İlk kolon bos oldugundan silinir
sched = sched.iloc[:,1:]
# converting from str to int
sched["FlNo"] = sched["FlNo"].astype("Int64",errors='ignore')
sched.sort_values(by=["Date","FlNo"],inplace=True)

In [8]:
# Convert time columns to datetime format
sched["STA"] = pd.to_datetime(sched['STA'].astype(str)).dt.time
sched["STD"] = pd.to_datetime(sched['STD'].astype(str)).dt.time

# taking difference in timedelta format
sched["bh"] = pd.to_datetime(sched["STA"].astype(str))-pd.to_datetime(sched["STD"].astype(str))

# Extract hours, minutes, and seconds from the time difference
sched["hour"] = sched["bh"].dt.components["hours"]
sched["minute"] = sched["bh"].dt.components["minutes"]
sched['second'] = sched['bh'].dt.components['seconds']

# Create a new BH column in "hour:minute:second" format
sched['bh'] = sched['hour'].astype(str).str.zfill(2) + ':' + \
                                 sched['minute'].astype(str).str.zfill(2) + ':' + \
                                 sched['second'].astype(str).str.zfill(2)
# convert dtype from str to datetime format
sched["bh"] = pd.to_datetime(sched.bh).dt.time

In [9]:
sched.bh

0       04:50:00
1       05:15:00
2       06:20:00
3       04:30:00
4       07:30:00
          ...   
1043    06:20:00
1044    05:35:00
1045    07:00:00
1046    06:30:00
1047    15:00:00
Name: bh, Length: 1048, dtype: object

In [10]:
# blok sürelerinin saat cinsinden ondalik hali
sched["BH"] = sched.Blkt.map(lambda x: x.hour+x.minute / 60)

In [11]:
aircraftList = list(sched.AC.unique())
aircraftList

['33X', '74A', '77X', 'ABW', '33K', '3RH']

## filter service type

In [12]:
sched.Srv.value_counts()

F    936
H     57
A     26
Z     17
Q      9
Y      3
Name: Srv, dtype: int64

In [13]:
sched = sched[sched["Srv"].isin(["F","H","A"])]
sched.reset_index(drop=True,inplace=True)

## Pick necessary columns

In [14]:
sched.columns

Index(['Al', 'FlNo', 'OS', 'Date', 'Day', 'Orig', 'STD', 'STA', 'over', 'Dest',
       'Own', 'AC', 'Cfg', 'Srv', 'Class', 'Blkt', 'Cntxt', 'Act', 'Change',
       'Time', 'By', 'bh', 'hour', 'minute', 'second', 'BH'],
      dtype='object')

In [15]:
sched = sched[["FlNo","Date","Day","Orig","STD","STA","over","Dest","Own","AC","Srv","bh","BH"]]

# Manipulation in DWH

In [16]:
dwh = pd.read_excel("dwh.xlsx")

In [17]:
dwh.columns

Index(['Leg Actual Departure GMT Year Month Day', 'Registration',
       'Actual Flight No', 'Flight Leg', 'Leg Actual Departure GMT OUT Time',
       'Leg Actual Departure GMT OFF Time', 'Leg Actual Arrival GMT ON Time',
       'Leg Actual Arrival GMT IN Time'],
      dtype='object')

In [18]:
dwh

,Leg Actual Departure GMT Year Month Day,Registration,Actual Flight No,Flight Leg,Leg Actual Departure GMT OUT Time,Leg Actual Departure GMT OFF Time,Leg Actual Arrival GMT ON Time,Leg Actual Arrival GMT IN Time
0,2023-01-01,LER,6277,ISTBCN,01:50,01:58,04:52,05:02
1,2023-01-01,LER,6278,BCNIST,06:47,07:14,10:19,10:27
2,2023-01-01,LER,6459,ISTTUN,23:05,23:15,01:21,01:30
3,2023-01-01,LER,6463,ISTBLL,12:10,12:23,15:23,15:30
4,2023-01-01,LER,6464,BLLIST,16:50,17:01,19:43,19:51
...,...,...,...,...,...,...,...,...
4673,2023-12-17,MRI,6156,LOSNBO,16:50,17:33,22:11,22:21
4674,2023-12-17,VEL,6092,ISTVNO,15:59,16:14,18:45,18:54
4675,2023-12-17,VEL,6093,VNOIST,20:26,20:36,22:50,22:58
4676,2023-12-17,VEL,6766,ISTBGW,03:23,03:34,05:53,06:00


In [19]:
dwh.rename(columns={'Leg Actual Departure GMT Year Month Day':"Date", 'Registration':"Reg",
       'Actual Flight No':"FlNo", 'Flight Leg':"leg", 'Leg Actual Departure GMT OUT Time':"aobt",
       'Leg Actual Departure GMT OFF Time':"depart", 'Leg Actual Arrival GMT ON Time':"landing",
       'Leg Actual Arrival GMT IN Time':"aibt"},inplace=True)

dwh = dwh[['Date', 'Reg', 'FlNo', 'leg', 'aobt',"depart","landing",'aibt']]
dwh.sort_values(by=["Date","FlNo"],inplace=True)

## CHECK Point 

In [20]:
dwh_enddate = dwh.Date.iloc[-1]
sched_startdate = sched.Date.iloc[0]

if dwh_enddate == sched_startdate :
    print(f"{dwh_enddate.date()} tarhinde kesistiler")
    intersection_day = dwh_enddate.date()
else:
    print("Tarihler kesismedi!!!")

2023-12-17 tarhinde kesistiler


In [21]:
intersection_day

datetime.date(2023, 12, 17)

## DWH ta BH sütunu oluşturma

In [22]:
dwh = dwh.copy()
# taking difference we get timedelta format
dwh["bh"] = pd.to_datetime(dwh["aibt"])-pd.to_datetime(dwh["aobt"])

# Extract hours, minutes, and seconds from the time difference
dwh["hour"] = dwh["bh"].dt.components["hours"]
dwh["minute"] = dwh["bh"].dt.components["minutes"]
dwh['second'] = dwh['bh'].dt.components['seconds']

# Create a new BH column in "hour:minute:second" format
dwh['bh'] = dwh['hour'].astype(str).str.zfill(2) + ':' + \
                                 dwh['minute'].astype(str).str.zfill(2) + ':' + \
                                 dwh['second'].astype(str).str.zfill(2)
# convert dtype from str to datetime format
dwh["bh"] = pd.to_datetime(dwh.bh).dt.time
dwh

,Date,Reg,FlNo,leg,aobt,depart,landing,aibt,bh,hour,minute,second
5,2023-01-01,VEL,6153,ISTVIE,17:12,17:22,19:13,19:22,02:10:00,2,10,0
6,2023-01-01,VEL,6154,VIEIST,20:43,20:54,22:37,22:44,02:01:00,2,1,0
0,2023-01-01,LER,6277,ISTBCN,01:50,01:58,04:52,05:02,03:12:00,3,12,0
1,2023-01-01,LER,6278,BCNIST,06:47,07:14,10:19,10:27,03:40:00,3,40,0
7,2023-01-01,VEL,6345,ISTBUD,03:26,03:35,05:08,05:24,01:58:00,1,58,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4676,2023-12-17,VEL,6766,ISTBGW,03:23,03:34,05:53,06:00,02:37:00,2,37,0
4677,2023-12-17,VEL,6767,BGWIST,07:20,07:39,10:21,10:30,03:10:00,3,10,0
4666,2023-12-17,LNX,6782,ISTALA,21:47,22:02,00:00,00:00,02:13:00,2,13,0
4667,2023-12-17,LNX,6783,ALAIST,11:40,12:29,18:20,18:52,07:12:00,7,12,0


In [23]:
dwh.drop(columns=["hour","minute","second"],inplace=True)

In [24]:
dwh["BH"] = dwh["bh"].apply(lambda x: x.hour +x.minute/60).map('{:.2f}'.format).astype(float)

## Son ay divert olanları listeden sil

In [25]:
last_day = dwh.Date.tail(1)
last_day

4665   2023-12-17
Name: Date, dtype: datetime64[ns]

In [26]:
last_month = last_day.dt.month

In [27]:
dwh = dwh[dwh.Date.dt.month == int(last_month)]
dwh.reset_index(drop=True,inplace=True)

kalkış ve gelis bacağı aynı olanları görelim:

In [28]:
aog = dwh["leg"].str[:3] == dwh["leg"].str[-3:]
aog.value_counts()

False    351
True       6
Name: leg, dtype: int64

In [29]:
dwh[aog]

,Date,Reg,FlNo,leg,aobt,depart,landing,aibt,bh,BH
25,2023-12-02,LER,6066,ISTIST,03:00,00:00,00:00,03:05,00:05:00,0.08
91,2023-12-05,LER,6115,ISTIST,12:35,00:00,00:00,12:45,00:10:00,0.17
131,2023-12-07,ACG,6275,ISTIST,07:09,00:00,00:00,08:05,00:56:00,0.93
150,2023-12-08,ACF,6227,NQZNQZ,07:25,00:00,00:00,07:52,00:27:00,0.45
229,2023-12-11,LNX,6787,ALAALA,16:08,00:00,00:00,17:04,00:56:00,0.93
345,2023-12-17,ACG,6403,ISTIST,02:38,02:53,03:54,04:02,01:24:00,1.40


In [30]:
print("Son ayda {} tane uçuştan {} tanesi orijine geri dönmüş.".format(len(dwh),len(dwh[aog])))

Son ayda 357 tane uçuştan 6 tanesi orijine geri dönmüş.


In [31]:
dwh = dwh[~aog]

## Havada olanları dwh tan sil

In [32]:
onair = dwh.aibt == "00:00"
# veya inis ve inblok time aynı ise
# onair = dwh.landing == dwh.aibt

In [33]:
len(dwh)

351

In [34]:
dwh[onair]

,Date,Reg,FlNo,leg,aobt,depart,landing,aibt,bh,BH
343,2023-12-17,ACF,6308,CDGIST,22:42,23:04,00:00,00:00,01:18:00,1.30
346,2023-12-17,ACG,6404,FRAIST,20:56,21:27,00:00,00:00,03:04:00,3.07
348,2023-12-17,MRH,6452,OSLIST,22:46,23:25,00:00,00:00,01:14:00,1.23
354,2023-12-17,LNX,6782,ISTALA,21:47,22:02,00:00,00:00,02:13:00,2.22


In [35]:
dwh = dwh[~onair]
dwh

,Date,Reg,FlNo,leg,aobt,depart,landing,aibt,bh,BH
0,2023-12-01,ACG,6060,ISTJFK,17:46,18:05,04:09,04:33,10:47:00,10.78
1,2023-12-01,MRH,6137,ISTCDG,00:41,00:57,04:26,04:40,03:59:00,3.98
2,2023-12-01,MRH,6138,CDGIST,22:21,22:45,01:21,01:28,03:07:00,3.12
3,2023-12-01,LER,6225,ALGLNZ,09:05,09:14,11:08,11:12,02:07:00,2.12
4,2023-12-01,LER,6225,ISTALG,04:21,04:33,07:54,08:05,03:44:00,3.73
...,...,...,...,...,...,...,...,...,...,...
351,2023-12-17,LER,6624,ISTBUD,22:01,22:11,23:36,23:42,01:41:00,1.68
352,2023-12-17,VEL,6766,ISTBGW,03:23,03:34,05:53,06:00,02:37:00,2.62
353,2023-12-17,VEL,6767,BGWIST,07:20,07:39,10:21,10:30,03:10:00,3.17
355,2023-12-17,LNX,6783,ALAIST,11:40,12:29,18:20,18:52,07:12:00,7.20


## Primary Keys

### Key for DWH  

Tesciller ucak tipine dönüştürülüp key oluşturulacak. \
Date + UCAKTIPI + FlNo + leg

In [36]:
dwh.Reg.unique()

array(['ACG', 'MRH', 'LER', 'ACF', 'MRI', 'VEL', 'LNX', 'LNY', 'LNZ',
       'LNW', 'LNV'], dtype=object)

In [37]:
dwh = dwh.reset_index().copy()

dwh["key"] = dwh["Date"].astype(str)+dwh["Reg"].map(dict_reg)+dwh["FlNo"].astype(str)+dwh["leg"]
# reorder columns
dwh = dwh[['key','Date', 'Reg', 'FlNo', 'leg', 'aobt', 'depart', 'landing', 'aibt', 'bh','BH',]]

In [38]:
dwh

,key,Date,Reg,FlNo,leg,aobt,depart,landing,aibt,bh,BH
0,2023-12-0174A6060ISTJFK,2023-12-01,ACG,6060,ISTJFK,17:46,18:05,04:09,04:33,10:47:00,10.78
1,2023-12-0133K6137ISTCDG,2023-12-01,MRH,6137,ISTCDG,00:41,00:57,04:26,04:40,03:59:00,3.98
2,2023-12-0133K6138CDGIST,2023-12-01,MRH,6138,CDGIST,22:21,22:45,01:21,01:28,03:07:00,3.12
3,2023-12-01ABW6225ALGLNZ,2023-12-01,LER,6225,ALGLNZ,09:05,09:14,11:08,11:12,02:07:00,2.12
4,2023-12-01ABW6225ISTALG,2023-12-01,LER,6225,ISTALG,04:21,04:33,07:54,08:05,03:44:00,3.73
...,...,...,...,...,...,...,...,...,...,...,...
342,2023-12-17ABW6624ISTBUD,2023-12-17,LER,6624,ISTBUD,22:01,22:11,23:36,23:42,01:41:00,1.68
343,2023-12-17ABW6766ISTBGW,2023-12-17,VEL,6766,ISTBGW,03:23,03:34,05:53,06:00,02:37:00,2.62
344,2023-12-17ABW6767BGWIST,2023-12-17,VEL,6767,BGWIST,07:20,07:39,10:21,10:30,03:10:00,3.17
345,2023-12-17RH6783ALAIST,2023-12-17,LNX,6783,ALAIST,11:40,12:29,18:20,18:52,07:12:00,7.20


### Key for SCHED

In [39]:
# Set date as column 
sched = sched.reset_index().copy()
# create key
sched["key"] = sched.Date.astype(str)+sched["AC"]+sched["FlNo"]+sched["Orig"]+sched["Dest"]
# reorder columns
sched = sched[['key','Date', 'FlNo', 'Orig', 'STD', 'STA', 'over', 'Dest',
       'AC', 'Srv', 'bh', 'BH']]

# Dwh ta olanları Sched ten silelim:

In [40]:
mevcut = len(sched)
mevcut

1019

In [41]:
common_rows = pd.merge(dwh, sched, how='inner', on=['key'])
common_rows

,key,Date_x,Reg,FlNo_x,leg,aobt,depart,landing,aibt,bh_x,...,FlNo_y,Orig,STD,STA,over,Dest,AC,Srv,bh_y,BH_y
0,2023-12-17ABW6092ISTVNO,2023-12-17,VEL,6092,ISTVNO,15:59,16:14,18:45,18:54,02:55:00,...,6092,IST,16:00:00,18:45:00,0,VNO,ABW,F,02:45:00,2.750000
1,2023-12-17ABW6093VNOIST,2023-12-17,VEL,6093,VNOIST,20:26,20:36,22:50,22:58,02:32:00,...,6093,VNO,20:45:00,23:35:00,0,IST,ABW,F,02:50:00,2.833333
2,2023-12-1733K6156ISTLOS,2023-12-17,MRI,6156,ISTLOS,07:51,08:11,15:12,15:27,07:36:00,...,6156,IST,07:30:00,14:25:00,0,LOS,33K,F,06:55:00,6.916667
3,2023-12-1733K6156LOSNBO,2023-12-17,MRI,6156,LOSNBO,16:50,17:33,22:11,22:21,05:31:00,...,6156,LOS,16:25:00,21:45:00,0,NBO,33K,F,05:20:00,5.333333
4,2023-12-1733K6452ISTSTN,2023-12-17,MRH,6452,ISTSTN,14:01,14:17,17:29,17:40,03:39:00,...,6452,IST,08:50:00,12:45:00,0,STN,33K,F,03:55:00,3.916667
5,2023-12-1733K6452STNOSL,2023-12-17,MRH,6452,STNOSL,19:07,19:28,21:02,21:04,01:57:00,...,6452,STN,14:45:00,16:55:00,0,OSL,33K,F,02:10:00,2.166667
6,2023-12-17ABW6624ISTBUD,2023-12-17,LER,6624,ISTBUD,22:01,22:11,23:36,23:42,01:41:00,...,6624,IST,16:00:00,18:10:00,0,BUD,ABW,A,02:10:00,2.166667
7,2023-12-17ABW6766ISTBGW,2023-12-17,VEL,6766,ISTBGW,03:23,03:34,05:53,06:00,02:37:00,...,6766,IST,03:15:00,06:15:00,0,BGW,ABW,H,03:00:00,3.000000
8,2023-12-17ABW6767BGWIST,2023-12-17,VEL,6767,BGWIST,07:20,07:39,10:21,10:30,03:10:00,...,6767,BGW,08:15:00,11:15:00,0,IST,ABW,H,03:00:00,3.000000


In [42]:
common_rows["key"].values

array(['2023-12-17ABW6092ISTVNO', '2023-12-17ABW6093VNOIST',
       '2023-12-1733K6156ISTLOS', '2023-12-1733K6156LOSNBO',
       '2023-12-1733K6452ISTSTN', '2023-12-1733K6452STNOSL',
       '2023-12-17ABW6624ISTBUD', '2023-12-17ABW6766ISTBGW',
       '2023-12-17ABW6767BGWIST'], dtype=object)

In [43]:
sched.drop(sched[sched["key"].isin(common_rows["key"].values)].index)
print("sched de {} tane data vardı, {} tane gerceklesen data silindi.".format(mevcut,len(common_rows)))
sched.reset_index(drop=True,inplace=True)

sched de 1019 tane data vardı, 9 tane gerceklesen data silindi.


In [44]:
dwh.Date

0     2023-12-01
1     2023-12-01
2     2023-12-01
3     2023-12-01
4     2023-12-01
         ...    
342   2023-12-17
343   2023-12-17
344   2023-12-17
345   2023-12-17
346   2023-12-17
Name: Date, Length: 347, dtype: datetime64[ns]

# Separate Wetlease companies

In [45]:
dwh.drop(columns="key",inplace=True)

sched_uls = sched[sched.AC == "ABW"]
sched_act = sched[sched.AC == "74A"]
sched_cma = sched[sched.AC == "33K"]

dwh_uls = dwh[dwh.Reg.isin(["VEL","LER"])]
dwh_act = dwh[dwh.Reg.isin(["ACG","ACF"])]
dwh_cma = dwh[dwh.Reg.isin(['MRG', 'MRH', 'MRI'])]

dwh_uls.reset_index(drop=True, inplace=True)
dwh_act.reset_index(drop=True, inplace=True)
dwh_cma.reset_index(drop=True, inplace=True)

In [46]:
dwh_uls 

,Date,Reg,FlNo,leg,aobt,depart,landing,aibt,bh,BH
0,2023-12-01,LER,6225,ALGLNZ,09:05,09:14,11:08,11:12,02:07:00,2.12
1,2023-12-01,LER,6225,ISTALG,04:21,04:33,07:54,08:05,03:44:00,3.73
2,2023-12-01,LER,6225,LNZIST,12:35,12:42,14:27,14:35,02:00:00,2.00
3,2023-12-01,VEL,6431,ISTBUD,14:08,14:16,15:52,15:58,01:50:00,1.83
4,2023-12-01,VEL,6432,BUDIST,18:10,18:20,19:50,19:56,01:46:00,1.77
...,...,...,...,...,...,...,...,...,...,...
121,2023-12-17,VEL,6093,VNOIST,20:26,20:36,22:50,22:58,02:32:00,2.53
122,2023-12-17,LER,6265,MXPIST,00:05,00:21,02:48,02:54,02:49:00,2.82
123,2023-12-17,LER,6624,ISTBUD,22:01,22:11,23:36,23:42,01:41:00,1.68
124,2023-12-17,VEL,6766,ISTBGW,03:23,03:34,05:53,06:00,02:37:00,2.62


In [47]:
dwh_act

,Date,Reg,FlNo,leg,aobt,depart,landing,aibt,bh,BH
0,2023-12-01,ACG,6060,ISTJFK,17:46,18:05,04:09,04:33,10:47:00,10.78
1,2023-12-01,ACF,6226,ALATPE,07:10,07:26,13:06,13:11,06:01:00,6.02
2,2023-12-01,ACF,6226,ISTALA,00:37,00:53,05:31,05:39,05:02:00,5.03
3,2023-12-01,ACF,6227,TPENQZ,16:55,17:18,00:51,01:05,08:10:00,8.17
4,2023-12-01,ACG,6276,ORDIST,02:29,02:47,12:23,12:32,10:03:00,10.05
...,...,...,...,...,...,...,...,...,...,...
65,2023-12-16,ACF,6060,ISTJFK,06:44,07:08,17:00,17:15,10:31:00,10.52
66,2023-12-16,ACF,6060,JFKORD,19:47,20:08,22:03,22:15,02:28:00,2.47
67,2023-12-17,ACF,6061,ORDIST,01:08,01:53,11:15,11:21,10:13:00,10.22
68,2023-12-17,ACF,6307,ISTCDG,15:09,15:38,18:44,19:04,03:55:00,3.92


In [48]:
dwh_cma

,Date,Reg,FlNo,leg,aobt,depart,landing,aibt,bh,BH
0,2023-12-01,MRH,6137,ISTCDG,00:41,00:57,04:26,04:40,03:59:00,3.98
1,2023-12-01,MRH,6138,CDGIST,22:21,22:45,01:21,01:28,03:07:00,3.12
2,2023-12-01,MRI,6332,ISTZRH,04:49,05:02,07:41,07:53,03:04:00,3.07
3,2023-12-01,MRI,6333,ZRHIST,10:14,10:32,12:42,12:49,02:35:00,2.58
4,2023-12-01,MRI,6351,ISTLOS,14:57,15:11,22:00,22:25,07:28:00,7.47
...,...,...,...,...,...,...,...,...,...,...
94,2023-12-17,MRI,6156,ISTLOS,07:51,08:11,15:12,15:27,07:36:00,7.60
95,2023-12-17,MRI,6156,LOSNBO,16:50,17:33,22:11,22:21,05:31:00,5.52
96,2023-12-17,MRH,6452,ISTSTN,14:01,14:17,17:29,17:40,03:39:00,3.65
97,2023-12-17,MRH,6452,STNOSL,19:07,19:28,21:02,21:04,01:57:00,1.95


In [49]:
sched_uls

,key,Date,FlNo,Orig,STD,STA,over,Dest,AC,Srv,bh,BH
12,2023-12-17ABW6092ISTVNO,2023-12-17,6092,IST,16:00:00,18:45:00,0,VNO,ABW,F,02:45:00,2.750000
13,2023-12-17ABW6093VNOIST,2023-12-17,6093,VNO,20:45:00,23:35:00,0,IST,ABW,F,02:50:00,2.833333
20,2023-12-17ABW6174ISTEBL,2023-12-17,6174,IST,03:30:00,06:00:00,0,EBL,ABW,F,02:30:00,2.500000
21,2023-12-17ABW6175EBLIST,2023-12-17,6175,EBL,07:30:00,10:10:00,0,IST,ABW,F,02:40:00,2.666667
65,2023-12-17ABW6624ISTBUD,2023-12-17,6624,IST,16:00:00,18:10:00,0,BUD,ABW,A,02:10:00,2.166667
...,...,...,...,...,...,...,...,...,...,...,...,...
959,2023-12-31ABW6093VNOIST,2023-12-31,6093,VNO,20:45:00,23:35:00,0,IST,ABW,F,02:50:00,2.833333
966,2023-12-31ABW6174ISTEBL,2023-12-31,6174,IST,03:30:00,06:00:00,0,EBL,ABW,F,02:30:00,2.500000
967,2023-12-31ABW6175EBLIST,2023-12-31,6175,EBL,07:30:00,10:10:00,0,IST,ABW,F,02:40:00,2.666667
1013,2023-12-31ABW6624ISTBUD,2023-12-31,6624,IST,16:00:00,18:10:00,0,BUD,ABW,A,02:10:00,2.166667


# Save as Excel file

In [50]:
from openpyxl import load_workbook

excel_file = 'dwh_classified.xlsx'
sheet_name1 = 'ULS'
sheet_name2 = "ACT"
sheet_name3 = "CMA"
# Load the entire workbook
book = load_workbook(excel_file)

# Open the specific writer
writer = pd.ExcelWriter(excel_file, engine='openpyxl') 

# Load existing sheets
writer.book = book

# Write DataFrame to active sheet
dwh_uls.to_excel(writer, sheet_name=sheet_name1, index=False)
dwh_act.to_excel(writer, sheet_name=sheet_name2, index=False)
dwh_cma.to_excel(writer, sheet_name=sheet_name3, index=False)

# Save the changes
writer.save()



C:\Users\E_SUER1\AppData\Local\Temp\ipykernel_21484\3653399118.py:14: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
C:\Users\E_SUER1\AppData\Local\Temp\ipykernel_21484\3653399118.py:22: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Hangi uçuşların hangi maksatla ayıklandığını ve data özetini içeren bir sheet olustur. 